# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [127]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [128]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,maun,-20.3240,23.6512,27.94,48,95,2.72,BW,1675022151
1,1,ketchikan,52.7463,-132.7516,6.00,60,13,8.05,CA,1675022156
2,2,palmas,-9.8302,-47.4392,29.61,56,74,1.67,BR,1675022157
3,3,bambous virieux,-30.5194,18.1414,17.71,49,0,1.26,ZA,1675022160
4,4,rikitea,1.5626,171.5894,27.00,81,91,9.41,KI,1675022161


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [129]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    s = 'Humidity'
)
    
# Display the map
humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [139]:
# Narrow down cities that fit criteria and drop any results with null values
filter1 = city_data_df[(city_data_df['Humidity'] < 80)]
filter2 = filter1[(filter1['Max Temp'] < 30)]
filter3 = filter2[(filter2['Wind Speed'] > 2)]
filterdf = filter3[(filter3['Cloudiness']<50)]
# Drop any rows with null values
# No nulls exist

# Display sample data
filterdf

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,ketchikan,52.7463,-132.7516,6.00,60,13,8.05,CA,1675022156
10,10,tezu,29.9923,0.5203,8.10,35,0,4.17,DZ,1675022177
15,15,santa maria,43.2646,146.3330,-4.30,42,0,3.38,JP,1675022201
22,22,arman,25.4685,80.9831,12.74,49,0,3.26,IN,1675022213
35,35,arraial do cabo,10.8618,74.8252,26.94,73,9,2.48,IN,1675022244
41,41,coquimbo,-31.3360,-70.4912,14.50,25,0,5.48,CL,1675022266
61,61,nanortalik,44.2753,108.4895,-9.52,51,0,6.09,MN,1675022330
64,64,beringovskiy,29.3005,123.7545,10.55,44,0,7.19,CN,1675022332
70,70,taree,13.0185,42.5369,22.86,77,6,11.28,ER,1675022371
74,74,locri,36.9512,96.5422,-5.76,19,0,2.77,CN,1675022388


### Step 3: Create a new DataFrame called `hotel_df`.

In [140]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filterdf[['City', 'Country', 'Lat','Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel
1,ketchikan,CA,52.7463,-132.7516,60,
10,tezu,DZ,29.9923,0.5203,35,
15,santa maria,JP,43.2646,146.3330,42,
22,arman,IN,25.4685,80.9831,49,
35,arraial do cabo,IN,10.8618,74.8252,73,
41,coquimbo,CL,-31.3360,-70.4912,25,
61,nanortalik,MN,44.2753,108.4895,51,
64,beringovskiy,CN,29.3005,123.7545,44,
70,taree,ER,13.0185,42.5369,77,
74,locri,CN,36.9512,96.5422,19,


In [132]:
for index, row in hotel_df.iterrows():
    print (row['Lng'])
    

0.5203
108.4324
-13.1275
-70.4912
96.5422
36.0655
102.3586
24.3844
102.9902
34.9863


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [141]:
# Set parameters to search for a hotel
radius = 10000
limit = 20


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    categories = 'accommodation.hotel'
    
    params = {
    "categories":categories,
    "filter":filters,
    "bias":bias,    
    "limit":limit,
    "apiKey":geoapify_key}
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ketchikan - nearest hotel: No hotel found
tezu - nearest hotel: No hotel found
santa maria - nearest hotel: No hotel found
arman - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
coquimbo - nearest hotel: No hotel found
nanortalik - nearest hotel: No hotel found
beringovskiy - nearest hotel: No hotel found
taree - nearest hotel: No hotel found
locri - nearest hotel: No hotel found
honningsvag - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
batangafo - nearest hotel: No hotel found
broken hill - nearest hotel: No hotel found
sawakin - nearest hotel: No hotel found
manokwari - nearest hotel: No hotel found
nabire - nearest hotel: No hotel found
lata - nearest hotel: No hotel found
gorkovskoye - nearest hotel: No hotel found
svetlogorsk - nearest hotel: No hotel found
emet - nearest hotel: Hostal Las Torres
keetmanshoop - nearest hotel: No hotel found
vauvert - nearest hotel: No hotel found
saint-francois - nearest

,City,Country,Lat,Lng,Humidity,Hotel,Hotel Name
1,ketchikan,CA,52.7463,-132.7516,60,,No hotel found
10,tezu,DZ,29.9923,0.5203,35,,No hotel found
15,santa maria,JP,43.2646,146.3330,42,,No hotel found
22,arman,IN,25.4685,80.9831,49,,No hotel found
35,arraial do cabo,IN,10.8618,74.8252,73,,No hotel found
41,coquimbo,CL,-31.3360,-70.4912,25,,No hotel found
61,nanortalik,MN,44.2753,108.4895,51,,No hotel found
64,beringovskiy,CN,29.3005,123.7545,44,,No hotel found
70,taree,ER,13.0185,42.5369,77,,No hotel found
74,locri,CN,36.9512,96.5422,19,,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [142]:
%%capture --no-display

# Configure the map plot

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    s = 'Humidity',
    hover_cols = ['Country','Hotel Name']
)

    
# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Country,Hotel Name)